In [10]:
# importing libraries
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot.pandas
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import time
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import MeanSquaredError
from pandas.tseries.offsets import DateOffset
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from bokeh.models.formatters import NumeralTickFormatter
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [11]:
all_values = pd.read_csv(Path("Resources/all_values.csv"))
all_values['date'] = pd.to_datetime(all_values['date'], format='%Y-%m')
all_values.set_index('date', inplace=True)

In [12]:
scaler_all = StandardScaler()
all_scaler = scaler_all.fit(all_values)
all_values_scaled = all_scaler.transform(all_values)

all_values_scaled_df = pd.DataFrame(all_values_scaled, columns=all_values.columns, index=all_values.index)
all_values_scaled_df.head()

,Composite,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2008-06-01,-1.120010,-0.664592,-0.452623,-0.981467,-0.848043,2.568481,-1.329134,-1.744835,-1.183779,-1.929083,0.208160,-0.678934,-1.897463,-1.378353,-1.491493,-1.190403,-1.408245
2008-07-01,-1.126250,-0.637850,-0.535303,-1.096815,-0.935423,2.568481,-1.280302,-1.689366,-1.099725,-1.901250,0.726901,-0.678934,-1.884047,-1.394462,-1.497273,-1.158831,-1.397238
2008-08-01,-1.131451,-0.605399,-0.435669,-0.975607,-0.862901,2.568481,-1.280302,-1.633897,-1.057698,-1.901250,0.402688,-0.795051,-1.763302,-1.346137,-1.503054,-1.179879,-1.375226
2008-09-01,-1.136131,-0.720345,-0.478985,-0.859073,-0.752012,2.568481,-1.270535,-1.550693,-1.116536,-1.776002,0.208160,-0.802309,-1.776718,-1.362245,-1.474150,-1.179879,-1.375226
2008-10-01,-1.152772,-0.881425,-1.320192,-1.192878,-1.010260,1.604616,-1.348667,-1.516025,-1.108130,-1.803835,-0.634795,-1.092603,-1.763302,-1.394462,-1.474150,-1.295643,-1.397238


In [13]:
y = all_values_scaled_df["Composite"]

In [14]:
X = all_values_scaled_df.drop(columns = "Composite")

In [15]:
y

date
2008-06-01   -1.120010
2008-07-01   -1.126250
2008-08-01   -1.131451
2008-09-01   -1.136131
2008-10-01   -1.152772
                ...   
2022-09-01    1.959140
2022-10-01    1.894135
2022-11-01    1.833290
2022-12-01    1.767765
2023-01-01    1.689760
Name: Composite, Length: 176, dtype: float64

In [16]:
X

,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,
2008-06-01,-0.664592,-0.452623,-0.981467,-0.848043,2.568481,-1.329134,-1.744835,-1.183779,-1.929083,0.208160,-0.678934,-1.897463,-1.378353,-1.491493,-1.190403,-1.408245
2008-07-01,-0.637850,-0.535303,-1.096815,-0.935423,2.568481,-1.280302,-1.689366,-1.099725,-1.901250,0.726901,-0.678934,-1.884047,-1.394462,-1.497273,-1.158831,-1.397238
2008-08-01,-0.605399,-0.435669,-0.975607,-0.862901,2.568481,-1.280302,-1.633897,-1.057698,-1.901250,0.402688,-0.795051,-1.763302,-1.346137,-1.503054,-1.179879,-1.375226
2008-09-01,-0.720345,-0.478985,-0.859073,-0.752012,2.568481,-1.270535,-1.550693,-1.116536,-1.776002,0.208160,-0.802309,-1.776718,-1.362245,-1.474150,-1.179879,-1.375226
2008-10-01,-0.881425,-1.320192,-1.192878,-1.010260,1.604616,-1.348667,-1.516025,-1.108130,-1.803835,-0.634795,-1.092603,-1.763302,-1.394462,-1.474150,-1.295643,-1.397238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,0.555394,0.933198,1.321821,1.307881,2.889769,2.391909,2.436150,2.396929,1.995344,1.180799,2.543332,2.288359,2.213786,1.687906,2.366695,2.245850
2022-10-01,0.431456,0.833182,1.274219,1.309241,3.532345,2.470041,2.498553,2.506200,2.009260,1.375327,2.666707,2.382272,2.213786,1.745713,2.461410,2.289875
2022-11-01,0.137055,1.147967,1.437160,1.495568,3.532345,2.518874,2.595624,2.590254,2.037093,0.986271,2.666707,2.516433,2.246002,1.803520,2.503506,2.333900


In [17]:
all_values_scaled_df.hvplot(
    xlabel='Date', 
    ylabel='Scaled Value', 
    title='Scaled Values',
    height=800,
    width=2000
)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [18]:
#training_begin = X.index.min()
#print(training_begin)
#training_end = X.index.min() + DateOffset(months=125)
#print(training_end)

In [19]:
#X_train = X.loc[training_begin:training_end]
#y_train = y.loc[training_begin:training_end]
#X_train.shape

In [20]:
#X_test = X.loc[training_end:]
#y_test = y.loc[training_end:]
#X_test.shape

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [22]:
# Neural network parameters
n_input_feats = len(X.columns)

n_outputs = 1

hidden_nodes_layer1 = 128
hidden_nodes_layer2 = 64
hidden_nodes_layer3 = 32
# hidden_nodes_layer4 = 4
print(f"""
Neural Network Parameters:
--------------------------
Input Features: {n_input_feats}
Layers:         3
Layer1 Nodes:   {hidden_nodes_layer1}
Layer2 Nodes:   {hidden_nodes_layer2}
Layer3 Nodes:   {hidden_nodes_layer3}
Output Nodes:   {n_outputs}
""")


Neural Network Parameters:
--------------------------
Input Features: 16
Layers:         3
Layer1 Nodes:   128
Layer2 Nodes:   64
Layer3 Nodes:   32
Output Nodes:   1



In [23]:
# define model
nn = Sequential()
# Add the first hidden layer
nn.add(Dense(
    units=hidden_nodes_layer1,
    input_dim = n_input_feats,
    activation='relu',
    name='hidden1'
))
# Add the second hidden layer
nn.add(Dense(
    units=hidden_nodes_layer2,
    activation='relu',
    name='hidden2'
))
# Add the third hidden layer
nn.add(Dense(
    units=hidden_nodes_layer3,
    activation='relu',
    name='hidden3'
))
# Add output
nn.add(Dense(
    n_outputs,
    activation='linear',
    name='output'
))

In [24]:
# Display the Sequential model summary
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden1 (Dense)             (None, 128)               2176      
                                                                 
 hidden2 (Dense)             (None, 64)                8256      
                                                                 
 hidden3 (Dense)             (None, 32)                2080      
                                                                 
 output (Dense)              (None, 1)                 33        
                                                                 
Total params: 12,545
Trainable params: 12,545
Non-trainable params: 0
_________________________________________________________________


In [25]:
# Compile the Sequential model
nn.compile(loss='mean_squared_error', optimizer='adam', metrics=[MeanSquaredError()])

In [26]:
# Fit the model 
model = nn.fit(
    X_train,
    y_train,
    batch_size=10,
    validation_split=0.2,
    epochs=100
)

Epoch 1/100
11/11 [==============================] - 2s 37ms/step - loss: 0.4170 - mean_squared_error: 0.4170 - val_loss: 0.0538 - val_mean_squared_error: 0.0538
Epoch 2/100
11/11 [==============================] - 0s 8ms/step - loss: 0.0565 - mean_squared_error: 0.0565 - val_loss: 0.1065 - val_mean_squared_error: 0.1065
Epoch 3/100
11/11 [==============================] - 0s 6ms/step - loss: 0.0483 - mean_squared_error: 0.0483 - val_loss: 0.0222 - val_mean_squared_error: 0.0222
Epoch 4/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0280 - mean_squared_error: 0.0280 - val_loss: 0.0190 - val_mean_squared_error: 0.0190
Epoch 5/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0172 - mean_squared_error: 0.0172 - val_loss: 0.0271 - val_mean_squared_error: 0.0271
Epoch 6/100
11/11 [==============================] - 0s 7ms/step - loss: 0.0101 - mean_squared_error: 0.0101 - val_loss: 0.0100 - val_mean_squared_error: 0.0100
Epoch 7/100
11/11 [==============

In [27]:
# R2 score for training
train_pred = nn.predict(X_train)
training_r2 = r2_score(y_train.values, train_pred)
training_r2

5/5 [==============================] - 0s 2ms/step


0.9990344893913224

In [28]:
train_predictions_df = X_train.copy()
train_predictions_df['Train Predictions'] = train_pred
cols = train_predictions_df.columns.tolist()
cols = [cols[-1]] + cols[:-1]
train_predictions_df = train_predictions_df[cols]

In [29]:
train_pred_descaled_df = pd.DataFrame(all_scaler.inverse_transform(train_predictions_df), columns=train_predictions_df.columns, index=train_predictions_df.index)
#train_pred_descaled_df['Actual'] = all_values['Composite'][:126].values
train_pred_descaled_df

,Train Predictions,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2013-02-01,394462.003843,399.80,47.132105,28.416316,22.750000,1.25,122.9,132.3,127.9,114.1,92.4,131.1,118.6,105.9,139.4,121.1,117.0
2012-02-01,381592.830008,275.40,40.709000,19.962500,13.996000,1.25,121.4,129.8,126.9,112.6,93.0,128.4,118.5,105.1,136.6,119.7,115.7
2015-03-01,450164.863613,296.30,54.639091,35.916818,27.362727,1.00,126.1,139.0,133.3,119.1,94.5,126.3,119.8,108.9,150.6,123.5,120.7
2011-03-01,357086.705075,323.80,48.085653,17.892609,13.270435,1.25,119.1,126.8,124.6,110.3,91.6,124.2,116.7,105.6,134.8,117.6,114.2
2009-12-01,332739.070836,241.00,39.199091,14.875909,11.698182,0.50,115.4,122.1,121.3,107.8,93.5,117.0,113.4,103.7,131.2,114.1,111.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-07-01,624548.461474,394.10,58.924091,41.976364,38.715909,2.00,136.4,150.8,144.3,123.4,95.7,142.3,127.9,119.2,170.8,133.6,130.8
2019-11-01,632359.285848,421.30,65.139500,45.670500,44.909500,2.00,136.8,151.7,145.9,123.6,97.1,142.8,128.0,115.8,172.1,133.9,131.1
2014-06-01,431354.059579,339.54,52.693333,32.176190,24.291905,1.25,125.4,135.9,132.2,116.1,93.7,131.3,118.8,107.4,146.5,123.3,118.9


In [30]:
# Create a DataFrame with the history dictionary
model_df = pd.DataFrame(model.history, index=range(1, len(model.history["loss"]) + 1))

In [31]:
model_df[['val_loss', 'mean_squared_error']].hvplot(
height=800,
    width=2000
)

:NdOverlay   [Variable]
   :Curve   [index]   (value)

In [32]:
predictions = nn.predict(X_test)

2/2 [==============================] - 0s 7ms/step


In [33]:
# R2 score from test
test_r2 = r2_score(y_test.values, predictions)
test_r2

0.9933129019432224

In [34]:
predictions_df = X_test.copy()
predictions_df['Test Predictions'] = predictions
cols = predictions_df.columns.tolist()
# Move the last column to the first position
cols = [cols[-1]] + cols[:-1]
# Reindex the DataFrame with the new column order
predictions_df = predictions_df[cols]
predictions_df.head()

,Test Predictions,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2016-04-01,-0.141966,-0.466135,-0.521209,-0.049976,-0.205649,-0.644402,-0.059487,0.217385,-0.124696,0.464539,0.402688,-0.468470,-0.072874,-0.025261,0.167575,-0.127483,0.000563
2018-12-01,0.386877,-0.347358,0.223305,-0.120347,-0.030080,0.962039,0.565570,0.515532,0.497305,0.798533,0.597216,0.380640,0.477186,0.812368,0.971096,0.577622,0.682954
2010-01-01,-1.015549,-0.664436,-0.912938,-1.146391,-1.033630,-0.965690,-1.241236,-1.259480,-1.225806,-1.400259,-0.569952,-1.143404,-1.119329,-1.185054,-1.294948,-1.232499,-1.210131
2014-03-01,-0.556405,-0.254307,-0.201224,-0.132433,-0.346894,-0.001826,-0.391549,-0.448244,-0.418886,-0.384362,-1.153536,-0.163662,-0.542437,-0.637374,-0.479866,-0.369534,-0.527740
2012-11-01,-0.721077,-0.330156,-0.842946,-0.542078,-0.599590,-0.001826,-0.625945,-0.600784,-0.746697,-0.634857,-1.218379,-0.374125,-0.502189,-0.717915,-0.878736,-0.632633,-0.725854


In [35]:
# Inverse transform for real values
pred_descaled_df = pd.DataFrame(all_scaler.inverse_transform(predictions_df), columns=predictions_df.columns, index=predictions_df.index)
pred_descaled_df

,Test Predictions,Lumber,Wood,XHB,ITB,Bank Rate,All-items 8,Food,Shelter,"Household operations, furnishings and equipment",Clothing and footwear,Transportation,Health and personal care,"Recreation, education and reading","Alcoholic beverages, tobacco products and recreational cannabis",All-items excluding food,All-items excluding food and energy
date,,,,,,,,,,,,,,,,,
2016-04-01,5.070713e+05,305.72,46.640476,34.352381,27.354286,0.75,127.9,143.5,134.9,121.5,94.3,126.9,121.9,110.7,156.4,125.0,123.1
2018-12-01,6.087643e+05,336.10,58.983158,33.145789,30.488421,2.00,134.3,147.8,142.3,123.9,94.6,138.6,126.0,115.9,170.3,131.7,129.3
2010-01-01,3.390871e+05,255.00,40.146316,15.553158,12.573684,0.50,115.8,122.2,121.8,108.1,92.8,117.6,114.1,103.5,131.1,114.5,112.1
2014-03-01,4.273775e+05,359.90,51.945238,32.938571,24.832857,1.25,124.5,133.9,131.4,115.4,91.9,131.1,118.4,106.9,145.2,122.7,118.3
2012-11-01,3.957121e+05,340.50,41.306666,25.914762,20.321905,1.25,122.1,131.7,127.5,113.6,91.8,128.2,118.7,106.4,138.3,120.2,116.5
2019-10-01,6.369451e+05,405.56,60.408261,44.517826,44.135652,2.00,136.7,151.5,145.7,123.6,96.9,142.2,127.8,117.3,171.4,133.9,131.4
2017-08-01,6.189042e+05,384.70,62.219565,38.087391,34.054348,1.00,130.4,143.5,138.0,122.0,94.0,132.0,124.7,113.5,161.7,127.9,125.6
2009-08-01,3.208370e+05,207.00,34.544285,15.134286,12.788571,0.50,114.5,121.4,121.1,107.1,92.9,114.4,112.2,103.3,131.4,113.1,111.6
2022-01-01,1.035548e+06,1329.00,90.376500,77.613499,74.124500,0.50,146.1,163.3,157.6,127.6,95.1,159.9,134.3,121.3,178.2,142.9,138.2


In [36]:
total_chart = pd.concat([train_pred_descaled_df, pred_descaled_df],axis=1) 
total_chart['Actual'] = all_values['Composite'].values

In [37]:
total_chart[['Actual', 'Train Predictions', 'Test Predictions']].hvplot(
    xlabel='Date', 
    ylabel='Price', 
    title='Actual and Predicted Index',
    height=800,
    width=2000
).opts(
    yformatter=NumeralTickFormatter(format="0,0")
)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [38]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def weighted_absolute_percentage_error(y_true, y_pred):
    return 100 / len(y_true) * np.sum(np.abs((y_true - y_pred) / y_true))

def weighted_mean_absolute_percentage_error(y_true, y_pred):
    return 100 / sum(y_true) * np.sum(np.abs(y_true - y_pred))

In [39]:
# Generate predictions
y_pred = nn.predict(X_test)

# Calculate R-Squared
r2 = r2_score(y_test, y_pred)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Absolute Percentage Error
mape = mean_absolute_percentage_error(y_test, y_pred)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)

# Calculate Root Mean Squared Error
rmse = np.sqrt(mse)

# Calculate Normalized Root Mean Squared Error
nrmse = rmse / (np.max(y_test) - np.min(y_test))

# Calculate Weighted Absolute Percentage Error
wape = weighted_absolute_percentage_error(y_test, np.squeeze(y_pred))

# Calculate Weighted Mean Absolute Percentage Error
wmape = weighted_mean_absolute_percentage_error(y_test, np.squeeze(y_pred))

2/2 [==============================] - 0s 3ms/step


In [40]:
print(f"R-Squared: {r2}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")
print(f"Normalized Root Mean Squared Error: {nrmse}")
print(f"Weighted Absolute Percentage Error: {wape}")
print(f"Weighted Mean Absolute Percentage Error: {wmape}")

R-Squared: 0.9933129019432224
Mean Absolute Error: 0.04145187547962281
Mean Absolute Percentage Error: 172.07812243762567
Mean Squared Error: 0.0060022770830948065
Root Mean Squared Error: 0.07747436403801458
Normalized Root Mean Squared Error: 0.020876970132266803
Weighted Absolute Percentage Error: 5.91062216251355
Weighted Mean Absolute Percentage Error: -70.75257229306862


In [42]:
nn_json = nn.to_json()

file_path = ("Models/model2.json")
with open(file_path, "w") as json_file:
    json_file.write(nn_json)

In [43]:
file_path = "Models/model2.h5"
nn.save_weights(file_path)

In [67]:
total_chart_save = pd.concat([train_pred_descaled_df['Train Predictions'], pred_descaled_df['Test Predictions']],axis=0) 
total_chart_save = total_chart_save.sort_index()
total_chart_save.rename("composite", inplace=True)
total_chart_save

date
2008-06-01    312415.493303
2008-07-01    315042.326405
2008-08-01    314801.977124
2008-09-01    314902.265927
2008-10-01    314692.427367
                  ...      
2022-09-01    907717.839914
2022-10-01    887167.139762
2022-11-01    885487.812349
2022-12-01    862896.332476
2023-01-01    864546.341173
Name: composite, Length: 176, dtype: float64

In [68]:
total_chart_save.to_csv('Resources/total_chart_save.csv', index=True)